In [3]:
from numba import cuda

In [11]:
@cuda.jit
def mat_mul(A, B, C):
    i, j = cuda.grid(2)
    if i < A.shape[0] and j < A.shape[0]:
        tmp = 0.
        for k in range(A.shape[0]):
            tmp += A[i, k] * B[k, j]
        C[i, j ] = tmp

In [8]:
import cupy as cp
cp.random.seed(42)
A = cp.random.uniform(1, 10, size=(200, 200), dtype=cp.float64)
B = cp.random.uniform(1, 10, size=(200, 200), dtype=cp.float64)
C = cp.zeros(shape=(200, 200), dtype=cp.float64)

In [9]:
threadsPerBlock = (16, 16)
blocksPerGrid_x = (A.shape[0] + (threadsPerBlock[0] - 1)) // threadsPerBlock[0]
blocksPerGrid_y = (A.shape[1] + (threadsPerBlock[1] - 1)) // threadsPerBlock[1]
blocksPerGrid = (blocksPerGrid_x, blocksPerGrid_y)
blocksPerGrid
print(f"Total number of execution goes upto element {blocksPerGrid_x * threadsPerBlock[0]}")

Total number of execution goes upto element 208


In [12]:
mat_mul[blocksPerGrid, threadsPerBlock](A, B, C)
C

array([[5504.27634676, 5522.6113957 , 5677.46175958, ..., 5861.19190738,
        5800.25156755, 5707.94379176],
       [5991.54234001, 5971.58079523, 6383.37636957, ..., 6297.86658399,
        6604.86029356, 6360.25295666],
       [5657.80959052, 5589.56466748, 6013.61936006, ..., 5983.42000211,
        6152.26032643, 5964.97501953],
       ...,
       [5706.09926741, 5688.72685426, 5778.03979915, ..., 6018.63701187,
        6090.61713863, 6041.59518873],
       [5633.2983972 , 5567.02072326, 5838.83593478, ..., 5946.83436965,
        6137.8581914 , 5845.79868409],
       [6248.11152538, 6288.84521256, 6711.20030092, ..., 6679.42713258,
        6870.18077904, 6692.0632367 ]])

In [14]:
from numba import float32, float64

TPB = 16

@cuda.jit
def fast_mul(A, B, C):
    # creating shared arrays
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    # get the absolute coordinates of a thread
    x, y = cuda.grid(2)

    # get the relative coordinates of a tread (relative to the block)
    tx, ty = cuda.threadIdx.x, cuda.threadIdx.y

    # get the x dimension of the grid => number of grids on the horizontal scale
    bpg = cuda.gridDim.x

    # checking of the current thread is well within the dimensions of the given array to be used in the multiplication
    if x >= C.shape[0] and y >= C.shape[1]:
        return      # ending function execution if the condition is not satisfied

    # Otherwise
    tmp = 0.
    # feed into the shared arrays the values from the matrices, A and B
    for i in range(bpg):
        sA[x, y] = A[x, ty + i * TPB]
        sB[x, y] = B[i * TPB + tx, y]

        # wait for all threads to finishing loading
        cuda.syncthreads()

        # performing dot multiplication
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # waiting for all threads to finish performing the dot multiplication
        cuda.syncthreads()

    C[x, y] = tmp


In [15]:
SIZE = 4000